In [ ]:
import json
import ollama
import numpy as np
import csv
from sklearn.cluster import DBSCAN
from tqdm import tqdm

# --- Configuración ---
JSON_FILE_PATH = '/scripts/LLM_Qdrant/Data/resultado_json.json'
CSV_OUTPUT_PATH = '/scripts/LLM_Qdrant/Data/analisis_grupos_tematicos.csv'
OLLAMA_HOST = 'http://ollama:11434'

# --- PARÁMETROS PARA EL CLUSTERING (AJUSTABLES) ---
DBSCAN_EPS = 0.4 
DBSCAN_MIN_SAMPLES = 2

# --- Modelos Sugeridos ---
EMBEDDING_MODEL_NAME = 'snowflake-arctic-embed2:latest'
GENERATIVE_MODEL_NAME = 'gemma3:latest' 

def clean_text(text):
    """Función para limpiar el texto de saltos de línea y espacios extra."""
    if not isinstance(text, str):
        return ""
    return ' '.join(text.replace('\r\n', ' ').replace('\n', ' ').strip().split())

def generate_cluster_summary(observations, client, model_name):
    """Usa un modelo generativo para crear un resumen temático de un grupo."""
    observations_list_str = "\n".join([f"- \"{obs}\"" for obs in observations])
    prompt = f"""
    Actúa como un analista policial experto.
    Tu tarea es leer la siguiente lista de observaciones y sintetizar el tema central o el evento principal que las une en una sola frase concisa y clara.
    No repitas frases completas. Enfócate en el núcleo del asunto.

    Observaciones:
    {observations_list_str}

    Tema central del grupo (en una sola frase concisa en español):
    """
    try:
        response = client.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}],
            options={'temperature': 0.1}
        )
        return response['message']['content'].strip()
    except Exception as e:
        print(f"  [!] Advertencia: No se pudo generar el resumen. Error: {e}")
        return "Error al generar resumen"

def main():
    """
    Función principal para procesar personas, agrupar sus observaciones,
    generar resúmenes y guardar todo en un CSV con barra de progreso.
    """
    print("--- INICIO DEL ANÁLISIS Y EXPORTACIÓN ---")
    
    # --- 1. Conexión a Ollama y carga de datos ---
    try:
        client = ollama.Client(host=OLLAMA_HOST)
        client.list() 
        print(f"Conexión con Ollama ({OLLAMA_HOST}) exitosa.")
    except Exception as e:
        print(f"Error: No se pudo conectar con Ollama. Detalle: {e}")
        return

    try:
        with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta: {JSON_FILE_PATH}")
        return

    person_ids = list(data.keys())
    total_observations_processed = 0

    # --- 2. Preparación y escritura en CSV ---
    try:
        with open(CSV_OUTPUT_PATH, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            # Escribimos la cabecera del CSV
            csv_writer.writerow([
                'id_persona', 
                'id_grupo_tematico', 
                'resumen_del_grupo', 
                'id_observacion_original', 
                'texto_observacion'
            ])

            # Usamos tqdm para la barra de progreso, iterando sobre cada persona
            for person_id in tqdm(person_ids, desc=f"Procesando {len(person_ids)} personas"):
                content = data[person_id]
                
                # --- 3. Recolección de observaciones para la persona actual ---
                all_observations_with_meta = []
                for key, value in content.items():
                    if key.startswith("observacion_") and isinstance(value, list):
                        obs_type = key.replace("observacion_", "").upper()
                        for index, obs_text in enumerate(value):
                            cleaned_obs = clean_text(obs_text)
                            if cleaned_obs:
                                all_observations_with_meta.append({
                                    "text": cleaned_obs,
                                    "type": obs_type,
                                    "original_index": index + 1,
                                    "composite_id": f"{person_id}-{obs_type}-{index + 1}"
                                })

                # Si no hay suficientes observaciones para comparar, se guardan como "sin grupo"
                if len(all_observations_with_meta) < DBSCAN_MIN_SAMPLES:
                    rows_to_write = []
                    for meta_info in all_observations_with_meta:
                        rows_to_write.append([
                            person_id,
                            f"{person_id}-SIN_GRUPO",
                            "Observación aislada (no se pudo agrupar)",
                            meta_info['composite_id'],
                            meta_info['text']
                        ])
                    if rows_to_write:
                        csv_writer.writerows(rows_to_write)
                        total_observations_processed += len(rows_to_write)
                    continue

                # --- 4. Generación de Embeddings y Clustering ---
                observations_texts = [obs['text'] for obs in all_observations_with_meta]
                try:
                    embeddings = [
                        res['embedding'] for res in (
                            client.embeddings(model=EMBEDDING_MODEL_NAME, prompt=obs) for obs in observations_texts
                        )
                    ]
                except Exception as e:
                    print(f"\nError al generar embeddings para Persona ID: {person_id}. Detalle: {e}. Saltando...")
                    continue

                clustering = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES, metric='cosine').fit(np.array(embeddings))
                labels = clustering.labels_
                
                # --- 5. Procesamiento de resultados y preparación para CSV ---
                rows_to_write = []
                unique_labels = sorted(set(labels))

                for cluster_id in unique_labels:
                    cluster_indices = np.where(labels == cluster_id)[0]
                    
                    if cluster_id == -1:
                        # Observaciones que no pertenecen a ningún grupo (ruido)
                        summary = "Observación sin grupo temático (ruido)"
                        group_id_str = f"{person_id}-SIN_GRUPO"
                    else:
                        # Observaciones que sí pertenecen a un grupo
                        observations_in_cluster = [observations_texts[i] for i in cluster_indices]
                        summary = generate_cluster_summary(observations_in_cluster, client, GENERATIVE_MODEL_NAME)
                        group_id_str = f"{person_id}-GRUPO-{cluster_id + 1}"

                    # Crear una fila en el CSV para cada observación en este grupo (o ruido)
                    for i in cluster_indices:
                        meta_info = all_observations_with_meta[i]
                        rows_to_write.append([
                            person_id,
                            group_id_str,
                            summary,
                            meta_info['composite_id'],
                            meta_info['text']
                        ])
                
                if rows_to_write:
                    csv_writer.writerows(rows_to_write)
                    total_observations_processed += len(rows_to_write)

    except Exception as e:
        print(f"\nHa ocurrido un error inesperado durante el proceso: {e}")
        return

    print("\n" + "="*60)
    print("--- PROCESO COMPLETADO ---")
    print(f"[✓] Se han analizado y guardado {total_observations_processed} observaciones de {len(person_ids)} personas.")
    print(f"Los resultados se han guardado en: {CSV_OUTPUT_PATH}")
    print("="*60)

if __name__ == '__main__':
    main()

--- INICIO DEL ANÁLISIS Y EXPORTACIÓN ---
Conexión con Ollama (http://ollama:11434) exitosa.


Procesando 3409 personas:   0%|          | 15/3409 [05:40<15:12:20, 16.13s/it]